In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import ee
import geemap.foliumap as geemap
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model

In [2]:
ee.Authenticate()
ee.Initialize(project="215656163750")
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
import utils
import explore

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# The fine-tuned model from initialModel.ipynb

model = load_model('/content/drive/MyDrive/CAFO_models/VGG16_step2.keras')

In [4]:
# Functions for data preparation

data_path = "/content/drive/MyDrive/CAFO_data/forTraining/"

def read_data(dataset):

  with open(f'{data_path}{dataset}_sentinel_images.pkl', 'rb') as f:
      image_dict = pickle.load(f)

  df = pd.read_pickle(f'{data_path}{dataset}_metadata_gdf.pkl')
  df["Sentinel"] = image_dict.values()

  return df


def prepare(df):

  images = []
  labels = []

  for idx, row in df.iterrows():
    if row['Farm type'] in ["Poultry", "Broiler", "Layer", "Pig", "Unknown CAFO"]:
      labels.append("CAFO")
      images.append(row["Sentinel"])
    elif row['Farm type'] == "Non-farm":
      labels.append("Non-farm")
      images.append(row["Sentinel"])
    else:
      print(f"Unexpected farm type, {row['Farm type']}")

  label_encoder = LabelEncoder()
  encoded_labels = label_encoder.fit_transform(labels)
  one_hot_labels = to_categorical(encoded_labels, num_classes=len(set(labels)))

  class_mapping = dict(zip(encoded_labels, labels))
  print("Class mapping:", class_mapping)

  return images, one_hot_labels

In [19]:
# Main function for finding images that need checking

def find_dodgy_images(dataset, min_prob=0.7):

  # Get the Sentinel images etc. for this country/location
  data = read_data(dataset)
  X, y = prepare(data)
  meta = data.drop(columns=["Sentinel"])

  # Apply the model to all images
  results = utils.get_predictions(model, X, y, meta)

  # Identify images labeled as not-farm but confidently predicted to be CAFOs
  results.loc[:, "CAFO prob"] = results.loc[:, "Model Probabilities"].str[0]
  to_check = results[(results["Label"] == 1) & (results["CAFO prob"] >= min_prob)]
  print(f"There are {len(to_check)} images to check")

  # Sort in descending probability order
  to_check.sort_values(by="CAFO prob", ascending=False, inplace=True)

  return data, to_check

## Mexico

In [20]:
mex, check_mex = find_dodgy_images("mex")

Class mapping: {0: 'CAFO', 1: 'Non-farm'}


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


210/210 ━━━━━━━━━━━━━━━━━━━━ 426s 2s/step
There are 213 images to check


<ipython-input-19-bf454368e365>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_check.sort_values(by="CAFO prob", ascending=False, inplace=True)


In [25]:
# Examine those images, flag ones that really are CAFOs

#mislabeled_1 = explore.loop_over_buildings(check_mex[:50])
#print(len(mislabeled_1))
#mislabeled_2 = explore.loop_over_buildings(check_mex[50:100])
#print(len(mislabeled_2))
#mislabeled_3 = explore.loop_over_buildings(check_mex[100:150])
#print(len(mislabeled_3))
mislabeled_4 = explore.loop_over_buildings(check_mex[150:])
print(len(mislabeled_4))

Working on feature 1 of 63


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Parent coords,geometry,CAFO prob
3521,1,"[0.7647572, 0.23524278]",0,México,Durango,Mapimí,2281.897949,11.126208,Mexico,Non-farm,160.382219,None,POINT (-104.0731 26.52756),0.764757


Enter reject to reject, exit to exit, or any key to continue  reject
Working on feature 2 of 63


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Parent coords,geometry,CAFO prob
802,1,"[0.7638631, 0.23613687]",0,México,Jalisco,Atotonilco el Alto,2507.5826,1.218003,Mexico,Non-farm,55.160007,None,POINT (-102.51417 20.63174),0.763863


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 3 of 63


...

Enter reject to reject, exit to exit, or any key to continue  
Working on feature 62 of 63


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Parent coords,geometry,CAFO prob
2425,1,"[0.70409685, 0.29590312]",0,México,Hidalgo,Huichapan,1056.0545,6.973522,Mexico,Non-farm,85.52883,None,POINT (-99.64378 20.36737),0.704097


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 63 of 63


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Parent coords,geometry,CAFO prob
12,1,"[0.70156425, 0.29843575]",0,México,México,Acambay de Ruíz Castañeda,846.608582,1.137633,Mexico,Non-farm,38.197306,NaN,POINT (-99.85821 19.89979),0.701564


Enter reject to reject, exit to exit, or any key to continue  
28


In [26]:
mex_mislabeled = mislabeled_1 + mislabeled_2 + mislabeled_3 + mislabeled_4
print(mex_mislabeled)

print(f"Rejected {len(mex_mislabeled)} / {len(check_mex)} images")

# Change the labels of the "bad" images in the original df, and pickle the
# "cleaned" df with a new name

df = mex.copy()
df.loc[df.index.isin(mex_mislabeled), ['Farm type']] = 'Unknown CAFO'

df.to_pickle(f'{data_path}mex_final.pkl')

[6, 1980, 1259, 4285, 5523, 2677, 4484, 4182, 2016, 4952, 3494, 4975, 4183, 4866, 6206, 3945, 6702, 1252, 4642, 3083, 210, 5062, 6372, 6222, 74, 6220, 1304, 3496, 2756, 2916, 3997, 148, 4079, 4867, 5654, 3998, 4298, 2357, 5663, 5972, 4234, 4078, 4868, 6218, 4604, 3174, 6377, 4488, 3365, 2464, 5107, 5232, 4937, 1138, 4621, 2678, 3675, 3082, 3523, 4862, 4132, 2100, 2353, 3748, 5229, 4546, 1261, 3999, 1358, 3239, 878, 71, 5658, 4861, 6240, 5574, 3524, 645, 5584, 3521, 3485, 3238, 3716, 6362, 914, 3456, 3243, 5534, 4415, 2915, 3515, 166, 5570, 6236, 5586, 3173, 2028, 3169, 590, 5575, 722, 72, 4863, 4916, 5691, 721, 3516]
Rejected 107 / 213 images


## Chile

In [27]:
chl, check_chl = find_dodgy_images("chl")

Class mapping: {1: 'Non-farm', 0: 'CAFO'}


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


21/21 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step
There are 18 images to check


<ipython-input-19-bf454368e365>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_check.sort_values(by="CAFO prob", ascending=False, inplace=True)


In [28]:
chl_mislabeled = explore.loop_over_buildings(check_chl)

Working on feature 1 of 18


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,ADM3,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Number of animals,Parent coords,CAFO prob
603,1,"[0.92804223, 0.07195783]",0,Chile,Región de Ñuble,Diguillín,Chillán,1471.270142,1.35173,Chile,Non-farm,47.579889,POINT (-72.19953 -36.61672),NaN,NaN,0.928042


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 2 of 18


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,ADM3,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Number of animals,Parent coords,CAFO prob
473,1,"[0.8821685, 0.11783156]",0,Chile,Región Metropolitana de Santiago,Maipo,Paine,2089.567139,1.351037,Chile,Non-farm,58.102094,POINT (-70.70312 -33.84452),NaN,None,0.882168


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 3 of 18


...

Enter reject to reject, exit to exit, or any key to continue  
Working on feature 17 of 18


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,ADM3,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Number of animals,Parent coords,CAFO prob
319,1,"[0.70420784, 0.29579216]",0,Chile,Región del Maule,Talca,Talca,1539.762451,8.092824,Chile,Non-farm,112.193256,POINT (-71.59308 -35.46068),NaN,None,0.704208


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 18 of 18


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,ADM3,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Number of animals,Parent coords,CAFO prob
444,1,"[0.7031826, 0.29681745]",0,Chile,Región Metropolitana de Santiago,Maipo,Buin,1871.7824,4.781655,Chile,Non-farm,101.255134,POINT (-70.71722 -33.73987),NaN,None,0.703183


Enter reject to reject, exit to exit, or any key to continue  


In [29]:
print(f"Rejected {len(chl_mislabeled)} / {len(check_chl)} images")

df = chl.copy()
df.loc[df.index.isin(chl_mislabeled), ['Farm type']] = 'Unknown CAFO'

df.to_pickle(f'{data_path}chl_final.pkl')

Rejected 2 / 18 images


## Iowa

In [30]:
iowa, check_iowa = find_dodgy_images("iowa")

Class mapping: {0: 'CAFO'}


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


98/98 ━━━━━━━━━━━━━━━━━━━━ 192s 2s/step
There are 0 images to check


<ipython-input-19-bf454368e365>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_check.sort_values(by="CAFO prob", ascending=False, inplace=True)


In [31]:
# There is nothing to check here, so we'll just write out the data df with a
# consistent name and format

iowa.to_pickle(f'{data_path}iowa_final.pkl')

## Romania


In [32]:
rou, check_rou = find_dodgy_images("rou")

Class mapping: {1: 'Non-farm', 0: 'CAFO'}


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step
There are 19 images to check


<ipython-input-19-bf454368e365>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_check.sort_values(by="CAFO prob", ascending=False, inplace=True)


In [33]:
rou_mislabeled = explore.loop_over_buildings(check_rou)

Working on feature 1 of 19


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Parent coords,CAFO prob
234,1,"[0.9977755, 0.002224435]",0,GALATI,1813.208955,1.477078,Romania,Non-farm,55.760327,POINT (28.12127 45.44932),NaN,0.997775


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 2 of 19


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Parent coords,CAFO prob
689,1,"[0.9952657, 0.00473425]",0,IZVORU BARZII,2181.708451,6.311394,Romania,Non-farm,118.130982,POINT (22.69409 44.66024),None,0.995266


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 3 of 19


...

Enter reject to reject, exit to exit, or any key to continue  
Working on feature 18 of 19


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Parent coords,CAFO prob
375,1,"[0.71094984, 0.28905022]",0,PECICA,1751.062537,3.683799,Romania,Non-farm,87.050315,POINT (21.01784 46.18279),NaN,0.71095


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 19 of 19


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Parent coords,CAFO prob
62,1,"[0.710131, 0.28986895]",0,SUDITI,1247.666173,3.022514,Romania,Non-farm,63.020964,POINT (27.56 44.57077),NaN,0.710131


Enter reject to reject, exit to exit, or any key to continue  


In [34]:
# Relabel the probably-CAFOs and write file

print(f"Rejected {len(rou_mislabeled)} / {len(check_rou)} images")

df = rou.copy()
df.loc[df.index.isin(rou_mislabeled), ['Farm type']] = 'Unknown CAFO'

df.to_pickle(f'{data_path}rou_final.pkl')

Rejected 7 / 19 images
